# Semantic region using Y@N data

We will use the user "184a19da-2ba4-4be0-bf31-7c03216a06b4" from our previous conclusions.

In [13]:
import pandas as pd
import gmaps
from wenet_models import LocationPoint, UserPlaceTimeOnly
from wenet_algo import estimate_stay_points, estimate_stay_regions, labelize_stay_region
from datetime import datetime

In [2]:
user = '184a19da-2ba4-4be0-bf31-7c03216a06b4'
df = pd.read_csv(f'/idiap/temp/wdroz/locations/{user}_location.csv')

In [3]:
len(df)

8268

In [4]:
df.head()

,userid,night,type,timestamp,timezone,local_time,source,latitude,longitude,speed,accuracy,provider,bearing
0,184a19da-2ba4-4be0-bf31-7c03216a06b4,20140919,Location,1411149633,7200,20140919200033,358661056383281,46.705295,6.557577,0.000000,3337.0,network,0.000000
1,184a19da-2ba4-4be0-bf31-7c03216a06b4,20140919,Location,1411149636,7200,20140919200036,358661056383281,46.696934,6.522324,4.000000,25.0,gps,40.000000
2,184a19da-2ba4-4be0-bf31-7c03216a06b4,20140919,Location,1411149647,7200,20140919200047,358661056383281,46.697126,6.522521,1.750000,34.0,gps,62.200001
3,184a19da-2ba4-4be0-bf31-7c03216a06b4,20140919,Location,1411149652,7200,20140919200052,358661056383281,46.696962,6.522723,3.000000,40.0,gps,75.699997
4,184a19da-2ba4-4be0-bf31-7c03216a06b4,20140919,Location,1411149657,7200,20140919200057,358661056383281,46.697171,6.523019,3.288237,21.0,gps,28.700001


In [5]:
locations = []
for index, row in df.iterrows():
    pts_t = datetime.fromtimestamp(row['timestamp'])
    location = LocationPoint(pts_t, row['latitude'], row['longitude'])
    locations.append(location)

In [6]:
stay_points = estimate_stay_points(locations, time_min_ms=5*60*100)
len(stay_points)

177

In [7]:
stay_regions = estimate_stay_regions(stay_points)
len(stay_regions)

5

In [8]:
df_ambiance = pd.read_csv('/idiap/temp/wdroz/wenet/surveys/surveys-2015-01-15/ambiance_survey.csv', sep=',', error_bad_lines=False, encoding="ISO-8859-1")
df_ambiance = df_ambiance[df_ambiance['user'] == user]

b'Skipping line 3: expected 19 fields, saw 20\nSkipping line 73: expected 19 fields, saw 20\nSkipping line 171: expected 19 fields, saw 20\nSkipping line 206: expected 19 fields, saw 20\nSkipping line 369: expected 19 fields, saw 20\nSkipping line 431: expected 19 fields, saw 20\nSkipping line 548: expected 19 fields, saw 20\nSkipping line 638: expected 19 fields, saw 20\nSkipping line 653: expected 19 fields, saw 20\nSkipping line 709: expected 19 fields, saw 20\nSkipping line 842: expected 19 fields, saw 20\nSkipping line 891: expected 19 fields, saw 20\nSkipping line 1034: expected 19 fields, saw 20\nSkipping line 1114: expected 19 fields, saw 20\nSkipping line 1172: expected 19 fields, saw 20\nSkipping line 1181: expected 19 fields, saw 20\nSkipping line 1368: expected 19 fields, saw 20\nSkipping line 1399: expected 19 fields, saw 20\nSkipping line 1400: expected 19 fields, saw 20\n'


In [9]:
df_ambiance.count()

user                         21
source                       21
timezone_id                  21
timestamp                    21
timezone_display_name        21
timezone_raw_offset          21
env_artsy_percent            21
env_dingy_percent            21
env_formal_percent           21
env_loud_percent             21
env_oldfashioned_percent     21
env_romantic_percent         21
env_sophisticated_percent    21
env_trendy_percent           21
env_upscale_percent          21
place_city                   21
place_id_name                21
place_type                   21
imageFileName                21
dtype: int64

In [10]:
df_ambiance.head()

,user,source,timezone_id,timestamp,timezone_display_name,timezone_raw_offset,env_artsy_percent,env_dingy_percent,env_formal_percent,env_loud_percent,env_oldfashioned_percent,env_romantic_percent,env_sophisticated_percent,env_trendy_percent,env_upscale_percent,place_city,place_id_name,place_type,imageFileName
97,184a19da-2ba4-4be0-bf31-7c03216a06b4,358661056383281,Europe/Zurich,2014-10-10 20:22:04,CET,7200000.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,other,other,plaza,IMG_20141010_202114_-1421440839.jpg
99,184a19da-2ba4-4be0-bf31-7c03216a06b4,358661056383281,Europe/Zurich,2014-10-10 23:33:47,CET,7200000.0,3.0,-1.0,0.0,3.0,1.0,0.0,1.0,2.0,0.0,other,other,bar,IMG_20141010_233139_-1421440839.jpg
100,184a19da-2ba4-4be0-bf31-7c03216a06b4,358661056383281,Europe/Zurich,2014-10-11 00:45:43,CET,7200000.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,other,public_transport,onboard,IMG_20141011_004443_-1421440839.jpg
101,184a19da-2ba4-4be0-bf31-7c03216a06b4,358661056383281,Europe/Zurich,2014-10-11 20:11:12,CET,7200000.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,lausanne,4c3e13fdb8b4be9a51a3cbef_Bella Vita,restaurant,IMG_20141011_200916_-872439340.jpg
103,184a19da-2ba4-4be0-bf31-7c03216a06b4,358661056383281,Europe/Zurich,2014-10-11 23:09:34,CET,7200000.0,1.0,0.0,2.0,1.0,1.0,2.0,3.0,-1.0,2.0,lausanne,5087e529e0e2e00927241ecd_Vapiano,restaurant,IMG_20141011_230827_-872439340.jpg


In [11]:
user_places = []
for index, row in df_ambiance.iterrows():
    pts_t = datetime.strptime(row['timestamp'], "%Y-%m-%d %H:%M:%S")
    if row['place_type'] == 'personal':
        place = row['place_id_name']
    else:
        place = row['place_type']
    user_place_time_only = UserPlaceTimeOnly(pts_t, place, user)
    user_place = user_place_time_only.to_user_place(locations)
    if user_place is not None:
        user_places.append(user_place)

In [14]:
labelize_stay_region(stay_regions, user_places)

set()

In [16]:
str(user_places[0])

'46.95659, 7.44765 [2014-10-10 20:22:04]'

In [19]:
for stay_region in stay_regions:
    print(stay_region)

46.52079, 6.63013[2014-09-20 21:28:45 to 2014-09-20 22:56:29]
46.69230, 6.52166[2014-10-04 21:03:18 to 2014-10-05 00:14:32]
46.46378, 6.84155[2014-09-26 21:08:16 to 2014-10-31 22:49:32]
46.52094, 6.63933[2014-10-18 21:02:37 to 2014-10-26 01:16:46]
46.51783, 6.63056[2014-09-20 23:35:32 to 2014-10-31 23:28:30]


In [22]:
all_stay_regions_center = [[s._lat, s._lng] for s in stay_regions]
all_stay_regions_center

[[46.52078853625, 6.63013085375],
 [46.692297736562494, 6.521657957656251],
 [46.46378174123377, 6.841545913181819],
 [46.520936324078946, 6.639334181842106],
 [46.51783276882353, 6.630556856764706]]

In [23]:
all_stay_regions_rect = [gmaps.Polygon([(s._topleft_lat, s._topleft_lng), (s._topleft_lat, s._bottomright_lng), (s._bottomright_lat, s._bottomright_lng), (s._bottomright_lat, s._topleft_lng)], fill_color='#FF8C00', stroke_color='#FF4500', stroke_weight=6) for s in stay_regions]

In [24]:
with open('google_api_key.txt', 'r') as f:
    key = f.read()
gmaps.configure(api_key=key)

In [26]:
raw_locations_df = df[['latitude', 'longitude']]
all_stay_regions_center_layer = gmaps.marker_layer(all_stay_regions_center)
all_stay_regions_rect_layer = gmaps.drawing_layer(features=all_stay_regions_rect)
stay_points_layer = gmaps.symbol_layer([[p._lat, p._lng] for p in stay_points], fill_color='#00FFFF', stroke_color='#00FFFF', scale=2)
place_points_layer = gmaps.symbol_layer([[p._lat, p._lng] for p in user_places], fill_color='#0000FF', stroke_color='#0000FF', scale=2)

locations_layer = gmaps.heatmap_layer(raw_locations_df)
fig = gmaps.figure()
fig.add_layer(stay_points_layer)
fig.add_layer(all_stay_regions_center_layer)
fig.add_layer(place_points_layer)
fig.add_layer(all_stay_regions_rect_layer)
fig.add_layer(locations_layer)

locations_layer.max_intensity = 100
locations_layer.point_radius = 10
fig.map_type = 'SATELLITE'
fig

Figure(layout=FigureLayout(height='420px'))